# In this part we will future explore the dataset

## Import libs

In [1]:
import pandas as pd

## load datasets

Load ad exposure log, Load ad_merge and test

In [2]:
df_exposure = pd.read_hdf('store_test.h5',key = 'df')

In [109]:
df_ad_info = pd.read_hdf('df_ad_merge.h5',key = 'df')

In [4]:
df_test = pd.read_hdf('df_testset.h5',key = 'df')

## Future explore

In [5]:
df_exposure.head()

,AdRequestId,Time,LocationId,UserId,Id,AdSize,Bid,pctr,quality_ecpm,total_ecpm
2,66156247,1550416600,18,808543,92253,40,96,3.824,76.48,443.584
25,39174722,1550406563,168,374668,109749,14,75,0.893,17.86,84.835
95,42177665,1550350462,297,1325591,717943,20,100,4.015,80.30,481.800
112,19971715,1550365077,247,63803,563411,40,40,8.048,160.96,482.880
180,17553277,1550380435,116,373844,433189,30,18,1.979,39.58,75.202


clean the expolog
+ drop duplicates
+ add "day" attribute

In [6]:
def exposure_clean(df_exposure):
    # before drop has 6005879 records
    temp = df_exposure.drop_duplicates(subset=['AdRequestId','Id'],keep = 'last')
    # after drop has 5952533 records
    # the target is exposure rate(daily), so we need to build an exposure rate attribute
    # first convert time to day
    temp_hours = pd.to_datetime(temp['Time'],unit='s') + pd.DateOffset(hours=8)
    temp_day = temp_hours.dt.date
    temp['day'] = temp_day
    return temp
expo_temp = exposure_clean(df_exposure)

/Users/yaoyao/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [7]:
def exposure_split_nochange(expo_temp,df_ad_info):
    # for these ads, no modification has done, so they can be directly used.
    df_ad_info_nochg = df_ad_info[df_ad_info['Date'].isnull()]
    expo_nochg = expo_temp[expo_temp['Id'].isin(set(df_ad_info_nochg['Id']))].sort_values('Id')
    df_id_expo = expo_nochg.groupby(['Id','day']).count().groupby('Id').mean().loc[:,'Time']
    # add daily exposure rate to the ad_info table
    df_ad_info_nochg['expo_rate'] = df_id_expo.to_list()
    df_ad_info_nochg[df_ad_info_nochg['Date_x'] == 0]
    return df_ad_info_nochg

    


In [8]:
df_ad_info_nochg = exposure_split_nochange(expo_temp,df_ad_info)

/Users/yaoyao/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [194]:
df_ad_info_nochg.shape

(2844, 13)

Next for the ads that have been modified several times
headache.....if an ad is modified several times in a day, then how to count the daily expo??? :(
my solution is to: separrate the ad change to bid, crow and expo
for ads only change bid, create an id called idbid, group them by idbid and date, 
calculate the the mean for every idbid pair.

In [191]:
def exposure_split_change_bid(expo_temp,df_ad_info):
    df_ad_info_chg = df_ad_info[~df_ad_info['Date'].isnull()]
    expo_chg = expo_temp[expo_temp['Id'].isin(set(df_ad_info_chg['Id']))].sort_values('Id')
    x1 = expo_chg[expo_chg['Id'].isin(set(df_ad_info_chg.loc[df_ad_info_chg['UpdateAttr'] == 3.0,'Id']))]
    x2 = expo_chg[expo_chg['Id'].isin(set(df_ad_info_chg.loc[df_ad_info_chg['UpdateAttr'] == 4.0,'Id']))]
    newset = set.union(set(x1['Id']),set(x1['Id']))
    expo_chg_bid = expo_chg[~expo_chg['Id'].isin(newset)]
    #expo_chg_bid = expo_chg
    expo_chg_bid['idbid'] = expo_chg_bid["Id"].map(str) + '_' + expo_chg_bid["Bid"].map(str)
    df_id_expo = expo_chg_bid.groupby(['idbid','day']).count().groupby('idbid').mean().loc[:,'Time']
    ad_chg_bid = df_ad_info_chg[~df_ad_info_chg['Id'].isin(newset)]
    ad_chg_bid['idbid'] = ad_chg_bid["Id"].map(str) + '_' + ad_chg_bid["Bid"].map(str)
    df_ad_info_chg_bid = pd.merge(ad_chg_bid,df_id_expo.to_frame().rename(columns={'Time':'expo_rate'}),on='idbid')
    return df_ad_info_chg_bid

In [192]:
df_ad_info_chg_bid = exposure_split_change_bid(expo_temp,df_ad_info)

/Users/yaoyao/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


In [195]:
df_ad_info_chg_bid.head()

,Id,Date_x,Date,Bid,Crowd,Time,UpdateAttr,AdAccountId,CommodityId,CommodityType,AdIndustryId,AdSize,idbid,expo_rate
0,32,1543563617,1550727091,91,area:7572,"70368475742208,70368475742208,70368475742208,7...",2.0,18752,32534,13,136,40,32_91,5.5
1,32,1543563617,1551110479,90,area:7572,"70368475742208,70368475742208,70368475742208,7...",2.0,18752,32534,13,136,40,32_90,3.0
2,32,1543563617,1551245501,96,area:7572,"70368475742208,70368475742208,70368475742208,7...",2.0,18752,32534,13,136,40,32_96,3.0
3,32,1543563617,1551331895,106,area:7572,"70368475742208,70368475742208,70368475742208,7...",2.0,18752,32534,13,136,40,32_106,3.0
4,32,1543563617,1551504704,97,area:7572,"70368475742208,70368475742208,70368475742208,7...",2.0,18752,32534,13,136,40,32_97,1.0


Now deal with the changes with crowd and expotime

In [202]:
def exposure_split_change_other(expo_temp,df_ad_info):
    df_ad_info_chg = df_ad_info[~df_ad_info['Date'].isnull()]
    expo_chg = expo_temp[expo_temp['Id'].isin(set(df_ad_info_chg['Id']))].sort_values('Id')
    x1 = expo_chg[expo_chg['Id'].isin(set(df_ad_info_chg.loc[df_ad_info_chg['UpdateAttr'] == 3.0,'Id']))]
    x2 = expo_chg[expo_chg['Id'].isin(set(df_ad_info_chg.loc[df_ad_info_chg['UpdateAttr'] == 4.0,'Id']))]
    newset = set.union(set(x1['Id']),set(x1['Id']))
    expo_chg_other = expo_chg[expo_chg['Id'].isin(newset)]
    ad_chg_other = df_ad_info_chg[df_ad_info_chg['Id'].isin(newset)]
    expo_chg_other['idbidday'] = expo_chg_other["Id"].map(str) + '_' + expo_chg_other["Bid"].map(str) +'_' + expo_chg_other["day"].map(str)
    
    temp_hours = pd.to_datetime(ad_chg_other['Date'],unit='s') + pd.DateOffset(hours=8)
    temp_day = temp_hours.dt.date
    ad_chg_other['day'] = temp_day
    ad_chg_other['idbidday'] = ad_chg_other["Id"].map(str) + '_' + ad_chg_other["Bid"].map(str) +'_' + ad_chg_other["day"].map(str)
    #expo_chg_bid['idbid'] = expo_chg_bid["Id"].map(str) + '_' + expo_chg_bid["Bid"].map(str)
    #df_id_expo = expo_chg_bid.groupby(['idbid','day']).count().groupby('idbid').mean().loc[:,'Time']
    
    #ad_chg_bid['idbid'] = ad_chg_bid["Id"].map(str) + '_' + ad_chg_bid["Bid"].map(str)
    #df_ad_info_chg_bid = pd.merge(ad_chg_bid,df_id_expo.to_frame().rename(columns={'Time':'expo_rate'}),on='idbid')
    return expo_chg_other, ad_chg_other

In [203]:
expo_chg_other, ad_chg_other = exposure_split_change_other(expo_temp,df_ad_info)

/Users/yaoyao/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/Users/yaoyao/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
/Users/yaoyao/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docume

In [201]:
ad_chg_other

,Id,Date_x,Date,Bid,Crowd,Time,UpdateAttr,AdAccountId,CommodityId,CommodityType,AdIndustryId,AdSize
139,764,1547629065,1551110443,10,"age:217,601,570,79,202,837,602,935,942,638,287...","281474976710655,281474976710655,28147497671065...",2.0,6655,-1,1,90,64
140,764,1547629065,1551110446,10,"age:217,601,570,79,202,837,602,935,942,638,287...","281474976710655,281474976710655,28147497671065...",3.0,6655,-1,1,90,64
286,1957,1550827427,1550856331,80,"age:829,79,847,144|area:10795,11414,10793|educ...","281474976710655,281474976710655,28147497671065...",2.0,12134,-1,1,104,41
287,1957,1550827427,1550940520,100,"age:829,79,847,144|area:10795,11414,10793|educ...","281474976710655,281474976710655,28147497671065...",2.0,12134,-1,1,104,41
288,1957,1550827427,1550966697,80,"age:829,79,847,144|area:10795,11414,10793|educ...","281474976710655,281474976710655,28147497671065...",3.0,12134,-1,1,104,41
290,2155,1552305999,1552338333,140,"age:217,601,202,942,638,394,347,731,739,393,36...","281474976710655,281474976710655,28147497671065...",2.0,27493,-1,1,2,20
291,2155,1552305999,1552340821,140,"age:217,601,202,942,638,394,347,731,739,393,36...","281474976710655,281474976710655,28147497671065...",3.0,27493,-1,1,2,20
292,2155,1552305999,1552340825,140,"age:217,601,202,942,638,394,347,731,739,393,36...","281474976710655,281474976710655,28147497671065...",3.0,27493,-1,1,2,20
293,2155,1552305999,1552388741,150,"age:217,601,202,942,638,394,347,731,739,393,36...","281474976710655,281474976710655,28147497671065...",2.0,27493,-1,1,2,20
294,2155,1552305999,1552391340,160,"age:217,601,202,942,638,394,347,731,739,393,36...","281474976710655,281474976710655,28147497671065...",2.0,27493,-1,1,2,20


In [69]:
df_ad_info_chg = df_ad_info[~df_ad_info['Date'].isnull()]
expo_chg = expo_temp[expo_temp['Id'].isin(set(df_ad_info_chg['Id']))].sort_values('Id')
x1 = expo_chg[expo_chg['Id'].isin(set(df_ad_info_chg.loc[df_ad_info_chg['UpdateAttr'] == 3.0,'Id']))]
len(set(x1['Id']))
x2 = expo_chg[expo_chg['Id'].isin(set(df_ad_info_chg.loc[df_ad_info_chg['UpdateAttr'] == 4.0,'Id']))]
len(set(x2['Id']))
newset = set.union(set(x1['Id']),set(x1['Id']))
len(newset)
#expo_chg_crow = expo_chg[~expo_chg['Id'].isin(newset)]

1486

2942

1486

In [115]:
tt1 = df_ad_info_chg.loc[df_ad_info_chg['UpdateAttr'] == 3.0,:]
temp_hours = pd.to_datetime(tt1['Date'],unit='s') + pd.DateOffset(hours=8)
temp_day = temp_hours.dt.date
tt1['day'] = temp_day
tt1['idbidday'] = tt1["Id"].map(str) + '_' + tt1["Bid"].map(str) +'_' + tt1["day"].map(str)
tt1[tt1['Id'] == 2155].sort_values('Date')

x1['idbidday'] = x1["Id"].map(str) + '_' + x1["Bid"].map(str) +'_' + x1["day"].map(str)
x1[x1['Id'] == 2155].sort_values('Time')

/Users/yaoyao/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Users/yaoyao/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


,Id,Date_x,Date,Bid,Crowd,Time,UpdateAttr,AdAccountId,CommodityId,CommodityType,AdIndustryId,AdSize,day,idbidday
291,2155,1552305999,1552340821,140,"age:217,601,202,942,638,394,347,731,739,393,36...","281474976710655,281474976710655,28147497671065...",3.0,27493,-1,1,2,20,2019-03-12,2155_140_2019-03-12
292,2155,1552305999,1552340825,140,"age:217,601,202,942,638,394,347,731,739,393,36...","281474976710655,281474976710655,28147497671065...",3.0,27493,-1,1,2,20,2019-03-12,2155_140_2019-03-12
299,2155,1552305999,1552477194,140,"age:217,601,202,942,638,394,347,731,739,393,36...","281474976710655,281474976710655,28147497671065...",3.0,27493,-1,1,2,20,2019-03-13,2155_140_2019-03-13
305,2155,1552305999,1552562220,140,"age:217,601,202,942,638,394,347,731,739,393,36...","281474976710655,281474976710655,28147497671065...",3.0,27493,-1,1,2,20,2019-03-14,2155_140_2019-03-14
330,2155,1552305999,1553002824,140,"age:217,601,202,942,638,394,347,731,739,393,36...","281474976710655,281474976710655,28147497671065...",3.0,27493,-1,1,2,20,2019-03-19,2155_140_2019-03-19
331,2155,1552305999,1553002830,140,"age:217,601,202,942,638,394,347,731,739,393,36...","281474976710655,281474976710655,28147497671065...",3.0,27493,-1,1,2,20,2019-03-19,2155_140_2019-03-19
332,2155,1552305999,1553002833,140,"age:217,601,202,942,638,394,347,731,739,393,36...","281474976710655,281474976710655,28147497671065...",3.0,27493,-1,1,2,20,2019-03-19,2155_140_2019-03-19
333,2155,1552305999,1553002834,140,"age:217,601,202,942,638,394,347,731,739,393,36...","281474976710655,281474976710655,28147497671065...",3.0,27493,-1,1,2,20,2019-03-19,2155_140_2019-03-19
334,2155,1552305999,1553002835,140,"age:217,601,202,942,638,394,347,731,739,393,36...","281474976710655,281474976710655,28147497671065...",3.0,27493,-1,1,2,20,2019-03-19,2155_140_2019-03-19
335,2155,1552305999,1553002836,140,"age:217,601,202,942,638,394,347,731,739,393,36...","281474976710655,281474976710655,28147497671065...",3.0,27493,-1,1,2,20,2019-03-19,2155_140_2019-03-19


/Users/yaoyao/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,AdRequestId,Time,LocationId,UserId,Id,AdSize,Bid,pctr,quality_ecpm,total_ecpm,day,idbidday
78450942,2393786,1552356143,297,295075,2155,20,140,8.193,163.86,1310.88,2019-03-12,2155_140_2019-03-12
75626778,5803387,1552356783,228,830594,2155,20,140,23.766,475.32,3802.56,2019-03-12,2155_140_2019-03-12
76974365,37555003,1552359370,297,16214,2155,20,140,3.199,63.98,511.84,2019-03-12,2155_140_2019-03-12
76737154,68056844,1552359901,297,209961,2155,20,140,5.963,119.26,954.08,2019-03-12,2155_140_2019-03-12
76049091,81635481,1552362596,228,382878,2155,20,160,14.300,286.00,2574.00,2019-03-12,2155_160_2019-03-12
77866520,10706731,1552364948,297,171968,2155,20,180,3.193,63.86,638.60,2019-03-12,2155_180_2019-03-12
75711064,84450572,1552364969,297,1107430,2155,20,180,6.137,122.74,1227.40,2019-03-12,2155_180_2019-03-12
77542187,20264944,1552365085,297,332500,2155,20,180,1.704,34.08,340.80,2019-03-12,2155_180_2019-03-12
77211295,31615934,1552365564,297,1059998,2155,20,180,5.035,100.70,1007.00,2019-03-12,2155_180_2019-03-12
76365853,38815426,1552365613,228,657517,2155,20,180,3.397,67.94,679.40,2019-03-12,2155_180_2019-03-12


In [172]:
tt1unique = tt1[tt1['Id'] == 2155].sort_values('Date').drop_duplicates(subset=['idbidday'],keep = 'last')

In [179]:
x1['idbidday'] = x1["Id"].map(str) + '_' + x1["Bid"].map(str) +'_' + x1["day"].map(str)
x1[x1['Id'] == 2155]

/Users/yaoyao/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,AdRequestId,Time,LocationId,UserId,Id,AdSize,Bid,pctr,quality_ecpm,total_ecpm,day,idbidday
77266749,31973200,1552366959,297,1188714,2155,20,180,4.748,94.96,949.60,2019-03-12,2155_180_2019-03-12
78080733,84826024,1552365837,297,598852,2155,20,180,7.619,152.38,1523.80,2019-03-12,2155_180_2019-03-12
88918114,591080,1552707272,297,379626,2155,20,200,3.584,71.68,788.48,2019-03-16,2155_200_2019-03-16
98235725,14110149,1552840238,297,859350,2155,20,150,4.222,84.44,717.74,2019-03-18,2155_150_2019-03-18
78468829,59728931,1552405876,228,1072514,2155,20,140,15.053,301.06,2408.48,2019-03-12,2155_140_2019-03-12
87850241,93757700,1552629365,297,1187936,2155,20,200,19.230,384.60,4230.60,2019-03-15,2155_200_2019-03-15
98709732,79186132,1552903721,297,786740,2155,20,160,3.004,60.08,540.72,2019-03-18,2155_160_2019-03-18
93732906,94510205,1552817296,297,1314142,2155,20,180,2.861,57.22,572.20,2019-03-17,2155_180_2019-03-17
100898338,44884369,1552963093,297,1247508,2155,20,150,1.045,20.90,177.65,2019-03-19,2155_150_2019-03-19
95781178,84269794,1552819638,297,652294,2155,20,180,3.390,67.80,678.00,2019-03-17,2155_180_2019-03-17


In [162]:
dates = pd.DataFrame({"day": pd.date_range(x1[x1['Id'] == 2155].day.min(), x1[x1['Id'] == 2155].day.max())})

In [173]:
dates = dates.astype(str)
tt1unique.day = tt1unique.day.astype(str)

In [178]:
t1 = pd.merge(tt1unique,dates,how = 'outer',on='day',sort=True)
t1.fillna(method='ffill')
#t2 = pd.concat([t1,tt1unique])

,Id,Date_x,Date,Bid,Crowd,Time,UpdateAttr,AdAccountId,CommodityId,CommodityType,AdIndustryId,AdSize,day,idbidday
0,2155.0,1.552306e+09,1552340825,140,"age:217,601,202,942,638,394,347,731,739,393,36...","281474976710655,281474976710655,28147497671065...",3.0,27493.0,-1,1.0,2,20.0,2019-03-12,2155_140_2019-03-12
1,2155.0,1.552306e+09,1552477194,140,"age:217,601,202,942,638,394,347,731,739,393,36...","281474976710655,281474976710655,28147497671065...",3.0,27493.0,-1,1.0,2,20.0,2019-03-13,2155_140_2019-03-13
2,2155.0,1.552306e+09,1552562220,140,"age:217,601,202,942,638,394,347,731,739,393,36...","281474976710655,281474976710655,28147497671065...",3.0,27493.0,-1,1.0,2,20.0,2019-03-14,2155_140_2019-03-14
3,2155.0,1.552306e+09,1552562220,140,"age:217,601,202,942,638,394,347,731,739,393,36...","281474976710655,281474976710655,28147497671065...",3.0,27493.0,-1,1.0,2,20.0,2019-03-15,2155_140_2019-03-14
4,2155.0,1.552306e+09,1552562220,140,"age:217,601,202,942,638,394,347,731,739,393,36...","281474976710655,281474976710655,28147497671065...",3.0,27493.0,-1,1.0,2,20.0,2019-03-16,2155_140_2019-03-14
5,2155.0,1.552306e+09,1552562220,140,"age:217,601,202,942,638,394,347,731,739,393,36...","281474976710655,281474976710655,28147497671065...",3.0,27493.0,-1,1.0,2,20.0,2019-03-17,2155_140_2019-03-14
6,2155.0,1.552306e+09,1552562220,140,"age:217,601,202,942,638,394,347,731,739,393,36...","281474976710655,281474976710655,28147497671065...",3.0,27493.0,-1,1.0,2,20.0,2019-03-18,2155_140_2019-03-14
7,2155.0,1.552306e+09,1553003802,140,"age:217,601,202,942,638,394,347,731,739,393,36...","281474976710655,281474976710655,28147497671065...",3.0,27493.0,-1,1.0,2,20.0,2019-03-19,2155_140_2019-03-19


In [167]:
#t2.drop_duplicates(subset='day',keep='last')

,day,0
0,2019-03-12,NaN
1,2019-03-13,NaN
2,2019-03-14,NaN
3,2019-03-19,NaN
4,2019-03-15,NaN
5,2019-03-16,NaN
6,2019-03-17,NaN
7,2019-03-18,NaN
342,NaN,2019-03-19


In [102]:
# open ad_static_feature file
collist = ['Id','Date','AdAccountId','CommodityId','CommodityType','AdIndustryId','AdSize']
with open('ad_static_feature.out') as f:
    df = pd.read_csv(f,sep='\t',names = collist)

/Users/yaoyao/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (3,6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [103]:
df[df['Id'] == 1957]

,Id,Date,AdAccountId,CommodityId,CommodityType,AdIndustryId,AdSize
108971,1957,1550827427,12134,-1,1,104,41


In [104]:
# open ad_operation file
collist2 = ['Id','Date','Type','UpdateAttr','UpdateAttrValue']
with open('ad_operation.dat') as f:
    df2 = pd.read_csv(f,sep='\t',names = collist2)

In [105]:
df2[df2['Id'] == 1957]


,Id,Date,Type,UpdateAttr,UpdateAttrValue
508986,1957,0,2,2,80
508987,1957,0,2,3,"age:829,79,847,144|area:10795,11414,10793|educ..."
508988,1957,0,2,4,"281474976710655,281474976710655,28147497671065..."
508989,1957,20190222172531,1,2,80
508990,1957,20190223164840,1,2,100
508991,1957,20190224000457,1,3,"age:829,79,847,144|area:10795,11414,10793|educ..."
508992,1957,20190225160301,1,1,0
